<a href="https://colab.research.google.com/github/waveology/aire/blob/main/acceso_a_ficheros_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# El acceso a datos almacenados en ficheros

A veces, el acceso a los datos almacenados en ficheros puede resultar tedioso. Con frecuencia, los datos no se encuentran en el formato que desearíamos y hay invertir algo de tiempo en incorporarlos a nuestros métodos de análisis.

###1. El origen de los datos
---

Vamos a ver un ejemplo basado en los datos de calidad del aire que publica la Comunidad de Madrid (España) y que están libremente disponibles en [su web](https://datos.comunidad.madrid/catalogo/dataset/calidad_aire_datos_historico)

Descargamos los datos de algún año del histórico:

In [ ]:
anio = 2023

In [ ]:
!rm -f {anio}.csv
!wget "https://datos.comunidad.madrid/catalogo/dataset/a770d92c-c513-4974-b1a7-2b15be1dd91f/resource/4aab1384-5abf-414e-9f42-af5fc334f436/download/{anio}.csv"

In [ ]:
a = "holas"
!echo "esto es $a asi"

El inventario del contenido puede consultarse [aquí](https://datos.comunidad.madrid/catalogo/dataset/a770d92c-c513-4974-b1a7-2b15be1dd91f/resource/f743eacc-5e89-4591-a0fc-4caebfe22557/download/descripcion-fichero-datos-de-contaminantes.pdf)

###2. Lectura de los datos
---

Empezamos por importar las extensiones que vamos a usar:

In [ ]:
import pandas as pd

A continuación, leemos los datos que están en formato CSV para generar un dataframe de Pandas. Especificamos que el separador de columnas es el punto y coma (;) y que el símbolo usado para designar decimales es la coma (,). El Ayuntamiento de Madrid usa el punto (.).

In [ ]:
df = pd.read_csv('%s.csv' % anio,  
                 sep=';', 
                 decimal=',')

Inspeccionamos la estructura del fichero:

In [ ]:
print(len(df))
print(df.head(5))

###3. El filtrado de columnas
---

Para empezar a filtrar, elegimos una estación de medida. Por ejemplo **Guadalix de la Sierra**, que según el inventario tiene:

 *   código : 28067**001** (el campo estación solo usa los 3 últimos dígitos) 
 *   municipio : 67

 
 Como magnitud elegimos el dióxido de nitrógeno (NO$_{2}$):

 *   magnitud : 8

In [ ]:
df = df[ (df['estacion']  == 1) 
       & (df['magnitud']  == 8) 
       & (df['municipio'] == 67)]
       
print(len(df))
print(df.head(5))          


###4. Eliminación de columnas no necesarias
---

Eliminamos la información que ya no necesitamos (provincia, municipio, estación, punto de muestreo y magnitud)

In [ ]:
df = df.drop(columns=['provincia','municipio','estacion','punto_muestreo','magnitud'])

print(df.head(5))     

###5. Pivotaje de filas y columnas
---

Nos gustaría tener en cada fila los datos correspondientes a cada instante. Sin embargo, la información de cada hora aparece en una columna. Podemos crear un nuevo dataframe en el que las horas aparezcan en una columna y la magnitud en otra: 

In [ ]:
df1 = df.melt(id_vars=['ano','mes','dia'],
                 value_vars = [ 'h%02d' % i for i in range(1,25)],
                 var_name='hora',
                 value_name='valor'
                 )
print(df1)

###6. Modificación y adaptación de valores
---

Pero la hora aparece en formato de texto con la letra 'h' delante.

Basta con eliminar la letra y convertir el resultado a valor numérico: 

In [ ]:
df1['hora'] = df1['hora'].apply(lambda x : int(x[1:]))

print(df1)

Repetimos la misma operación con los valores de validez de los datos:

In [ ]:
df2 = df.melt(id_vars=['ano','mes','dia'],
                 value_vars = [ 'v%02d' % i for i in range(1,25)],
                 var_name='hora',
                 value_name='flag'
                 )
df2['hora'] = df2['hora'].apply(lambda x : int(x[1:]))
print(df2)

###7. Fusión de tablas
---

A continuación fusionamos ambos dataframes, el que contiene las magnitudes y el que tiene los flags de calidad del dato:

In [ ]:
df = df1.merge(df2)
print(df)

###8. Filtrado por filas
---

Seleccionamos solo los datos válidos. Después eliminamos la columna 'flag', que yo no es necesaria. 

In [ ]:
n_antes = len(df)

df = df[df['flag'] == 'V'].drop(columns='flag')
print(df)

n_despues = len(df)
print('Eliminados %d datos malos' % (n_antes - n_despues))

###9. El tiempo
---

Ahora nos vendría muy bien que el tiempo estuviera en una única columna que incluya la fecha y la hora (datetime):

In [ ]:
df['fecha'] = pd.to_datetime({'year':df.ano,'month':df.mes,'day':df.dia,'hour':df.hora})
print(df)

Ya no necesitamos las columnas de año, mes, día y hora:

In [ ]:
df = df.drop(columns=['ano','mes','dia','hora'])
print(df)

###10. Reordenar columnas si es necesario
---

Por lo general no es necesario reordenar las columnas pero podemos hacerlo así:

In [ ]:

df = df[['fecha','valor']]
print(df)

###11. Resultado final
---

Ahora nuestros datos están dispuestos en un formato que simplifica muchas tareas de análisis:

In [ ]:
df.plot(x='fecha',y='valor')

Podemos embellecer el gráfico añadiendo parámetros a la función plot:

In [ ]:
df.plot(x='fecha',y='valor',
        fontsize=12,
        figsize=(15,10),
        marker='o',
        ms=5,
        lw=1,
        grid=True,
        legend=False
        )